In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!sudo apt install build-essential

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [3]:
%pip install -r "/content/drive/MyDrive/Protein-binding/requirements.txt"
%pip install datasets mdtraj dssp
%pip install numpy --no-cache-dir
%pip install pandas==2.2.0

ERROR: Could not find a version that satisfies the requirement dssp (from versions: none)
ERROR: No matching distribution found for dssp


In [4]:
import torch
import ast
import torch.nn as nn
import pandas as pd
import numpy as np
import torch.nn.functional as F
import warnings
import mdtraj as md
import esm
import gc
import pickle
import torch.optim as optim

from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool, global_add_pool
from Bio import SeqIO
from Bio import PDB
from Bio.PDB.DSSP import DSSP
from Bio.PDB.Polypeptide import is_aa
from Bio.SeqUtils import seq1
from torch_geometric.data import Data
from transformers import (AutoModelForTokenClassification, AutoTokenizer,
                          AutoModelForMaskedLM, DataCollatorForTokenClassification,
                           EsmForMaskedLM, EsmTokenizer, EsmModel, EsmForTokenClassification,
                           TrainingArguments, Trainer, TrainerCallback
                        )
from transformers.trainer_callback import ProgressCallback
from sklearn.metrics import (accuracy_score, precision_recall_fscore_support,
                             matthews_corrcoef, roc_auc_score)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from pprint import pprint
from datasets import Dataset
from datetime import datetime
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
from glob import glob
from loguru import logger

warnings.filterwarnings("ignore", message="Ignoring unrecognized record 'END'")

### Preparing train-test dataset

In [5]:
initial_train_df = pd.read_csv("/content/drive/MyDrive/Protein-binding/data/development_set/full_grouped_train_binding_sites_df.csv")
initial_train_df['binding_sites'] = initial_train_df['binding_sites'].apply(ast.literal_eval)
initial_train_df['any_ligand_binding_sites'] = initial_train_df['any_ligand_binding_sites'].apply(ast.literal_eval)

In [6]:
initial_train_df

,prot_id,binding_sites,ligand_type,sequence,sequence_length,metal_binding,small_binding,nuclear_binding,any_ligand_binding_sites
0,A0A0F7RDM3,"[29, 53, 129, 135, 137]",['metal'],MEIRKKLVVPSKYGTKCPYTMKPKYITVHNTYNDAPAENEVNYMIT...,234,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,A0A0F7RHX8,"[67, 72, 76, 78, 123, 133, 135]",['small'],MHLKEKITTIIQGQRTGVLSTVRNDKPHSAFMMFFHEDFVLYVATD...,138,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,A0A0H2W6Y8,"[85, 119, 125, 126, 129, 144, 197, 198, 207, 2...",['metal'],MMKILGLIGGMSWESTIPYYRMINQHVKAQLGGLHSAKIILYSVDF...,231,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,A0A2B6C3P9,"[79, 111, 114, 117, 122, 125]",['metal'],MSINKWLFRFIGFLVMLVVITTLNSLNVFASVNDLAQPIASAKVIE...,129,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,A0A384LKY8,"[5, 10, 12, 14, 43, 63, 72, 74, 76, 83, 85, 11...","['metal', 'small']",MSEHFVGKYEVELKFRVMDLTTLHEQLVAQKATAFTLNNHEKDIYL...,179,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, ..."
...,...,...,...,...,...,...,...,...,...
1009,S0BAP9,"[56, 66, 69, 70, 72, 73, 77, 80, 81, 86, 89, 9...",['small'],MNPLSTVLLVLCATSAALASEFCSEADATIVIKQWNQIYNAGIGAK...,168,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1010,S3TFW2,"[75, 79, 122, 192, 193, 204, 277, 278, 279, 28...",['small'],MTRPDSKSMNYQLLKTFSRQPIQFGRFLARLLAGLVNTLKITRTSK...,327,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1011,V6F235,"[230, 232, 255]",['small'],MRKSGCAVCSRSIGWVGLAVSTVLMVMKAFVGLIGGSQAMLADAMY...,318,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1012,V9P0A9,"[13, 33, 34, 37, 38, 42, 49, 51, 70, 72, 74, 7...",['small'],MADFKFEPMRSLIYVDCVSEDYRPKLQRWIYKVHIPDSISQFEPYV...,283,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [7]:
np.random.seed(42)
excluded_protein_id = ['Q9NZV6']

train_df = initial_train_df[~initial_train_df['prot_id'].isin(excluded_protein_id)]

# train_df, val_df = train_test_split(
#     needed_training_sites_df,
#     test_size = 0.1,  # 90% train, 10% validation
#     stratify = None,
#     random_state = 42
# )

In [8]:
test_df = pd.read_csv("/content/drive/MyDrive/Protein-binding/data/development_set/full_grouped_test_binding_sites_df.csv")
test_df['binding_sites'] = test_df['binding_sites'].apply(ast.literal_eval)
test_df['any_ligand_binding_sites'] = test_df['any_ligand_binding_sites'].apply(ast.literal_eval)

In [9]:
def split_into_chunks(sequences, labels, chunk_size = 1000):
    """Split sequences and labels into chunks of size "chunk_size" or less."""
    new_sequences = []
    new_labels = []
    for seq, lbl in zip(sequences, labels):
        if len(seq) > chunk_size:
            # Split the sequence and labels into chunks of size "chunk_size" or less
            for i in range(0, len(seq), chunk_size):
                new_sequences.append(seq[i:i+chunk_size])
                new_labels.append(lbl[i:i+chunk_size])
        else:
            new_sequences.append(seq)
            new_labels.append(lbl)

    return new_sequences, new_labels

In [10]:
# Initial sequences
test_seq = test_df['sequence'].tolist()
test_labels = test_df['any_ligand_binding_sites'].tolist()

train_seq = train_df['sequence'].tolist()
train_labels = train_df['any_ligand_binding_sites'].tolist()

# val_seq = val_df['sequence'].tolist()
# val_labels = val_df['any_ligand_binding_sites'].tolist()

# Apply new sequences by chunking
chunk_size = 1000

test_seq, test_labels = split_into_chunks(test_seq, test_labels, chunk_size)
train_seq, train_labels = split_into_chunks(train_seq, train_labels, chunk_size)
# val_seq, val_labels = split_into_chunks(val_seq, val_labels, chunk_size)

### Tokenization and get embeddings from ESM-2 language model

In [11]:
pretrained_model = "facebook/esm2_t33_650M_UR50D"

tokenizer = EsmTokenizer.from_pretrained(pretrained_model)
max_sequence_length = 1000

train_tokenized = tokenizer(train_seq, padding=True, truncation=True, max_length=max_sequence_length, return_tensors="pt", is_split_into_words=False)
test_tokenized = tokenizer(test_seq, padding=True, truncation=True, max_length=max_sequence_length, return_tensors="pt", is_split_into_words=False)
# eval_tokenized = tokenizer(val_seq, padding=True, truncation=True, max_length=max_sequence_length, return_tensors="pt", is_split_into_words=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [12]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [13]:
def get_embeddings_list(tokenized_dataset, batch_size, return_hidden_states = True, return_attentions = False):
    ids_list = torch.tensor(tokenized_dataset['input_ids']).to(device)
    attention_mask_list = torch.tensor(tokenized_dataset['attention_mask']).to(device)

    esm2 = EsmModel.from_pretrained(pretrained_model).to(device)

    num_batches = len(ids_list) // batch_size + (len(ids_list) % batch_size > 0)
    embeddings = []

    for i in tqdm(range(num_batches), total = num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(ids_list))

        batch_ids = ids_list[start_idx:end_idx]
        batch_attention_mask = attention_mask_list[start_idx:end_idx]

        with torch.no_grad():
            outputs = esm2(input_ids=batch_ids,
                           attention_mask=batch_attention_mask,
                           output_hidden_states = return_hidden_states,
                           output_attentions = return_attentions)

            hidden_states = outputs.hidden_states
            last_hidden_state = hidden_states[-1]

            batch_embeddings = []
            for j in range(last_hidden_state.shape[0]):
                mask = batch_attention_mask[j].bool()
                seq_embeddings = last_hidden_state[j][mask]
                seq_embeddings = seq_embeddings[1:-1]
                batch_embeddings.append(seq_embeddings)

            embeddings.extend(batch_embeddings)

    return embeddings

In [14]:
gc.collect()  # Force garbage collection to potentially free up memory
torch.cuda.empty_cache()  # Empty the CUDA cache

In [15]:
# train_embeddings = get_embeddings_list(train_tokenized, 8)
# test_embeddings = get_embeddings_list(test_tokenized, 8)
# # eval_embeddings = get_embeddings_list(eval_tokenized, 8)

In [16]:
# with open('/content/drive/MyDrive/Protein-binding/train_embeddings.pkl', 'wb') as f:
#        pickle.dump(train_embeddings, f)

# with open('/content/drive/MyDrive/Protein-binding/test_embeddings.pkl', 'wb') as f:
#     pickle.dump(test_embeddings, f)

In [17]:
with open('/content/drive/MyDrive/Protein-binding/train_embeddings.pkl', 'rb') as f:
       train_embeddings = pickle.load(f)

with open('/content/drive/MyDrive/Protein-binding/test_embeddings.pkl', 'rb') as f:
    test_embeddings = pickle.load(f)

### Features extraction

In [18]:
def get_structure(prot_id, pdb_file):
    parser = PDB.PDBParser()
    structure = parser.get_structure(prot_id, pdb_file)
    return structure

def extract_coordinates(structure):
    # Extract Cα coordinates (central carbon atom)
    coordinates = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if "CA" in residue:  # Get Cα atom
                    ca_atom = residue["CA"]
                    coord = ca_atom.get_coord()  # Returns numpy array [x, y, z]
                    coordinates.append(coord)

    return coordinates

def get_amino_acid_types(structure):
    amino_acids = []
    for model in structure:
        for chain in model:
            for residue in chain:
                # Get residue name (3-letter code)
                if not is_aa(residue):
                    continue
                res_name = residue.get_resname()
                # Convert to 1-letter code if needed
                one_letter = seq1(res_name)
                amino_acids.append(one_letter)
    return amino_acids

def get_secondary_structure_mdtraj(pdb_file, sequence_length):
    """
    Extract secondary structure features from a .PDB file using mdtraj.

    Args:
        pdb_file (str): Path to the .PDB file.
        sequence_length (int): Length of the protein sequence.

    Returns:
        dict: A dictionary with:
            - 'raw': List of raw secondary structure codes.
            - 'one_hot': Tensor of one-hot encoded secondary structure.
    """
    # Load the .PDB file with mdtraj
    traj = md.load(pdb_file)

    # Compute secondary structure
    ss = md.compute_dssp(traj)[0]  # Returns codes like 'H', 'E', 'C', '-'('NA')

    # One-hot encode
    ss_onehot = []
    for code in ss:
        ss_onehot.append([
            1 if code == 'H' else 0,
            1 if code == 'E' else 0,
            1 if code == 'T' else 0,
            1 if code == 'C' or code == 'NA' else 0
        ])

    ss_onehot = torch.tensor(ss_onehot, dtype=torch.float32)

    # Pad with zeros to match sequence length
    if ss_onehot.shape[0] < sequence_length:
        padding = torch.zeros((sequence_length - ss_onehot.shape[0], ss_onehot.shape[1]), dtype=torch.float32)
        ss_onehot = torch.cat([ss_onehot, padding], dim=0)

    return {
        "raw": ss.tolist(),
        "one_hot": ss_onehot
    }

def calculate_residue_distances(coordinates):
    """
    Calculate pairwise distances between residues in a protein structure.
    Args: Coordinates (list): List of residue atom's coordinates.
    Returns:
        np.ndarray: 2D array of pairwise distances.
    """
    num_residues = len(coordinates)
    distances = np.zeros((num_residues, num_residues))

    for i in range(num_residues):
        for j in range(i + 1, num_residues):
            dist = np.linalg.norm(coordinates[i] - coordinates[j])
            distances[i, j] = distances[j, i] = dist

    return distances

In [19]:
# def get_dihedral_angles(pdb_file):
#     traj = md.load(pdb_file)
#     # Compute phi and psi angles
#     phi_indices, phi_angles = md.compute_phi(traj)
#     psi_indices, psi_angles = md.compute_psi(traj)

#     # Convert to degrees and create tensors
#     phi_angles = torch.tensor(np.degrees(phi_angles[0]), dtype=torch.float32).unsqueeze(1)  # Shape: [num_residues-1, 1]
#     psi_angles = torch.tensor(np.degrees(psi_angles[0]), dtype=torch.float32).unsqueeze(1)  # Shape: [num_residues-1, 1]

#     # Pad with zeros for the first/last residue (since phi/psi are not defined for terminal residues)
#     #Adjusted Padding to maintain dimension
#     phi_angles = torch.cat([torch.zeros(1, 1, dtype=torch.float32), phi_angles], dim=0)
#     psi_angles = torch.cat([torch.zeros(1, 1, dtype=torch.float32), psi_angles], dim=0)

#     return phi_angles, psi_angles

# def get_chi_angles(pdb_file):
#     traj = md.load(pdb_file)
#     chi_indices, chi_angles = md.compute_chi1(traj)
#     chi_angles = torch.tensor(np.degrees(chi_angles[0]), dtype=torch.float32).unsqueeze(1)  # Shape: [num_residues, 1]
#     # Pad for residues without chi1 angles (e.g., glycine)
#     chi_angles_padded = torch.zeros((traj.n_residues, 1), dtype=torch.float32)
#     for i, idx in enumerate(chi_indices[:, 0]):  # Map to residue indices
#         chi_angles_padded[idx] = chi_angles[i]
#     return chi_angles_padded

# def get_b_factors(structure):
#     b_factors = []
#     for model in structure:
#         for chain in model:
#             for residue in chain:
#                 if "CA" in residue:
#                     ca_atom = residue["CA"]
#                     b_factor = ca_atom.get_bfactor()
#                     b_factors.append(b_factor)
#     return torch.tensor(b_factors, dtype=torch.float32).unsqueeze(1)  # Shape: [num_residues, 1]

In [20]:
def get_dihedral_angles(pdb_file, sequence_length):
    traj = md.load(pdb_file)
    # Compute phi and psi angles
    phi_indices, phi_angles = md.compute_phi(traj)
    psi_indices, psi_angles = md.compute_psi(traj)

    # Convert to degrees and create tensors
    phi_angles = torch.tensor(np.degrees(phi_angles[0]), dtype=torch.float32).unsqueeze(1)  # Shape: [num_residues-1, 1]
    psi_angles = torch.tensor(np.degrees(psi_angles[0]), dtype=torch.float32).unsqueeze(1)  # Shape: [num_residues-1, 1]

    # Pad with zeros to match sequence length
    while phi_angles.shape[0] < sequence_length:
        phi_angles = torch.cat([phi_angles, torch.zeros(1, 1, dtype=torch.float32)], dim=0)
    while psi_angles.shape[0] < sequence_length:
        psi_angles = torch.cat([psi_angles, torch.zeros(1, 1, dtype=torch.float32)], dim=0)

    return phi_angles, psi_angles

def get_chi_angles(pdb_file, sequence_length):
    traj = md.load(pdb_file)
    chi_indices, chi_angles = md.compute_chi1(traj)
    chi_angles = torch.tensor(np.degrees(chi_angles[0]), dtype=torch.float32).unsqueeze(1)  # Shape: [num_residues, 1]

    # Pad for residues without chi1 angles (e.g., glycine) to match sequence length
    chi_angles_padded = torch.zeros((sequence_length, 1), dtype=torch.float32)
    for i, idx in enumerate(chi_indices[:, 0]):  # Map to residue indices
        if idx < sequence_length:
            chi_angles_padded[idx] = chi_angles[i]

    return chi_angles_padded

def get_b_factors(structure, sequence_length):
    b_factors = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if "CA" in residue:
                    ca_atom = residue["CA"]
                    b_factor = ca_atom.get_bfactor()
                    b_factors.append(b_factor)

    # Pad with zeros if b_factors length is less than sequence length
    while len(b_factors) < sequence_length:
        b_factors.append(0.0)

    return torch.tensor(b_factors, dtype=torch.float32).unsqueeze(1)

In [21]:
def fuse_features(esm2_embeddings, ss_onehot, phi_angles, psi_angles, b_factors):
    """
    Fuse ESM-2 embeddings with structural features.

    Args:
        esm2_embeddings (torch.Tensor): Shape [num_residues, 1280]
        ss_onehot (torch.Tensor): Shape [num_residues, 4]
        phi_angles (torch.Tensor): Shape [num_residues, 1]
        psi_angles (torch.Tensor): Shape [num_residues, 1]
        b_factors (torch.Tensor): Shape [num_residues, 1]

    Returns:
        torch.Tensor: Fused node features, shape [num_residues, 1287] (1280 + 4 + 1 + 1 + 1)
    """
    # Ensure all features have the same length
    num_residues = esm2_embeddings.shape[0]

    # Print shapes for debugging
    print(f"esm2_embeddings shape: {esm2_embeddings.shape}")
    print(f"ss_onehot shape: {ss_onehot.shape}")
    print(f"phi_angles shape: {phi_angles.shape}")
    print(f"psi_angles shape: {psi_angles.shape}")
    print(f"b_factors shape: {b_factors.shape}")

    # Adjust ss_onehot, phi_angles, psi_angles, b_factors if needed
    min_length = min(num_residues, ss_onehot.shape[0], phi_angles.shape[0], psi_angles.shape[0], b_factors.shape[0])

    esm2_embeddings = esm2_embeddings[:min_length]
    ss_onehot = ss_onehot[:min_length]
    phi_angles = phi_angles[:min_length]
    psi_angles = psi_angles[:min_length]
    b_factors = b_factors[:min_length]

    # Concatenate all features
    node_features = torch.cat([
        esm2_embeddings,  # [num_residues, 1280]
        ss_onehot,        # [num_residues, 4]
        phi_angles,       # [num_residues, 1]
        psi_angles,       # [num_residues, 1]
        b_factors,        # [num_residues, 1]
    ], dim=1)  # Shape: [num_residues, 1287]

    return node_features

In [22]:
# def create_edge_features(distances, threshold=8.0):
#     num_residues = distances.shape[0]
#     contact_map = (distances < threshold) & (distances > 0)
#     edge_index = torch.nonzero(torch.tensor(contact_map, dtype=torch.bool), as_tuple=False).t()  # Shape: [2, num_edges]

#     # Vectorized edge attribute computation
#     src, dst = edge_index[0], edge_index[1]
#     dists = torch.tensor(distances[src, dst], dtype=torch.float32)
#     seq_seps = torch.abs(src - dst).float()

#     # Normalize features
#     dists = dists / threshold  # Scale to 0–1
#     seq_seps = seq_seps / num_residues  # Scale to 0–1

#     edge_attr = torch.stack([dists, seq_seps], dim=1)  # Shape: [num_edges, 2]

#     return edge_index, edge_attr

def create_edge_features(distances, threshold=8.0):
    num_residues = distances.shape[0]
    contact_map = (distances < threshold) & (distances > 0)

    # Restrict indices to be within the valid node range (0 to num_residues - 1)
    edge_index = torch.nonzero(torch.tensor(contact_map, dtype=torch.bool), as_tuple=False).t()
    edge_index = edge_index[:, :num_residues]  # Ensure indices are within bounds

    # Vectorized edge attribute computation
    src, dst = edge_index[0], edge_index[1]
    dists = torch.tensor(distances[src, dst], dtype=torch.float32)
    seq_seps = torch.abs(src - dst).float()

    # Normalize features
    dists = dists / threshold  # Scale to 0–1
    seq_seps = seq_seps / num_residues  # Scale to 0–1

    edge_attr = torch.stack([dists, seq_seps], dim=1)  # Shape: [num_edges, 2]

    return edge_index, edge_attr


In [23]:
def create_graph_data(node_features, edge_index, edge_attr, labels):
    """
    Create a PyTorch Geometric Data object for the protein graph.

    Args:
        node_features (torch.Tensor): Shape [num_residues, 1288], fused node features.
        edge_index (torch.Tensor): Shape [2, num_edges], indices of connected nodes.
        edge_attr (torch.Tensor): Shape [num_edges, 2], edge features.
        labels (torch.Tensor): Shape [num_residues], binary labels (0 or 1) for binding sites.

    Returns:
        Data: PyTorch Geometric Data object.
    """
    return Data(
        x=node_features,      # Node features
        edge_index=edge_index,  # Edge indices
        edge_attr=edge_attr,  # Edge features
        y=labels              # Labels for binding site prediction
    )

### Test feature extraction

In [24]:
# sample_idx = 540
# sample_prot_id = train_df.iloc[sample_idx]['prot_id']
# sample_sequence = train_df.iloc[sample_idx]['sequence']
# sample_sequence_len = len(train_df.iloc[sample_idx]['sequence'])
# sample_labels = train_df.iloc[sample_idx]['any_ligand_binding_sites']
# sample_train_embeddings = train_embeddings[sample_idx]

# sample_structure_file = f"/content/drive/MyDrive/Protein-binding/esmFold_pdb_files/{sample_prot_id}.pdb"

# sample_structure = get_structure(sample_prot_id, sample_structure_file)
# sample_coordinates = extract_coordinates(sample_structure)
# sample_distances = calculate_residue_distances(sample_coordinates)
# sample_ss_one_hot = get_secondary_structure_mdtraj(sample_structure_file, sample_sequence_len)['one_hot'].to(device)
# sample_phi_angles, sample_psi_angles = get_dihedral_angles(sample_structure_file, sample_sequence_len)
# sample_b_factors = get_b_factors(sample_structure, sample_sequence_len).to(device)

# print(sample_prot_id)
# print(sample_sequence)
# print(len(sample_sequence))
# print(sample_b_factors.shape)
# print(sample_ss_one_hot.shape)
# print(sample_phi_angles.shape)
# print(sample_psi_angles.shape)

In [25]:
# edge_index, edge_attr = create_edge_features(sample_distances)
# node_features = fuse_features(sample_train_embeddings.to(device), sample_ss_one_hot,
#                               sample_phi_angles.to(device), sample_psi_angles.to(device), sample_b_factors)
# labels = torch.tensor(sample_labels, dtype = torch.long).to(device)

In [26]:
# graph_data = create_graph_data(node_features, edge_index, edge_attr, labels)

In [27]:
# print(len(train_df))
# print(len(train_labels))
# print(len(train_seq))

### Prepare data tensors

In [28]:
def get_graph_data(df, embeddings, device):
    graph_data_list = []

    for idx in tqdm(range(len(df))):
        prot_id = df.iloc[idx]['prot_id']
        sequence = df.iloc[idx]['sequence']
        sequence_len = len(df.iloc[idx]['sequence'])
        labels = df.iloc[idx]['any_ligand_binding_sites']
        embedding = embeddings[idx]

        try:
            structure_file = f"/content/drive/MyDrive/Protein-binding/esmFold_pdb_files/{prot_id}.pdb"
            structure = get_structure(prot_id, structure_file)
            coordinates = extract_coordinates(structure)
            distances = calculate_residue_distances(coordinates)
            ss_one_hot = get_secondary_structure_mdtraj(structure_file, sequence_len)['one_hot'].to(device)
            phi_angles, psi_angles = get_dihedral_angles(structure_file, sequence_len)
            b_factors = get_b_factors(structure, sequence_len).to(device)
        except:
            print(f"PDB file of {prot_id} ID cannot be found")

        edge_index, edge_attr = create_edge_features(distances)
        node_features = fuse_features(embedding.to(device), ss_one_hot,
                                    phi_angles.to(device), psi_angles.to(device), b_factors)
        labels = torch.tensor(labels, dtype = torch.long).to(device)
        print(f"Shape of labels: {labels.shape}")
        graph_data = create_graph_data(node_features, edge_index, edge_attr, labels)

        graph_data_list.append(graph_data)

    return graph_data_list

In [29]:
train_graphs_data = get_graph_data(train_df, train_embeddings, device)
test_graphs_data = get_graph_data(test_df, test_embeddings, device)

  0%|          | 2/1013 [00:00<02:49,  5.98it/s]

esm2_embeddings shape: torch.Size([234, 1280])
ss_onehot shape: torch.Size([234, 4])
phi_angles shape: torch.Size([234, 1])
psi_angles shape: torch.Size([234, 1])
b_factors shape: torch.Size([234, 1])
Shape of labels: torch.Size([234])
esm2_embeddings shape: torch.Size([138, 1280])
ss_onehot shape: torch.Size([138, 4])
phi_angles shape: torch.Size([138, 1])
psi_angles shape: torch.Size([138, 1])
b_factors shape: torch.Size([138, 1])
Shape of labels: torch.Size([138])


  0%|          | 3/1013 [00:00<03:08,  5.36it/s]

esm2_embeddings shape: torch.Size([231, 1280])
ss_onehot shape: torch.Size([231, 4])
phi_angles shape: torch.Size([231, 1])
psi_angles shape: torch.Size([231, 1])
b_factors shape: torch.Size([231, 1])
Shape of labels: torch.Size([231])
esm2_embeddings shape: torch.Size([129, 1280])
ss_onehot shape: torch.Size([129, 4])
phi_angles shape: torch.Size([129, 1])
psi_angles shape: torch.Size([129, 1])
b_factors shape: torch.Size([129, 1])
Shape of labels: torch.Size([129])


  1%|          | 7/1013 [00:00<02:03,  8.16it/s]

...

esm2_embeddings shape: torch.Size([146, 1280])
ss_onehot shape: torch.Size([146, 4])
phi_angles shape: torch.Size([146, 1])
psi_angles shape: torch.Size([146, 1])
b_factors shape: torch.Size([146, 1])
Shape of labels: torch.Size([146])
esm2_embeddings shape: torch.Size([180, 1280])
ss_onehot shape: torch.Size([180, 4])
phi_angles shape: torch.Size([180, 1])
psi_angles shape: torch.Size([180, 1])
b_factors shape: torch.Size([180, 1])
Shape of labels: torch.Size([180])


100%|█████████▉| 299/300 [01:13<00:00,  5.02it/s]

esm2_embeddings shape: torch.Size([131, 1280])
ss_onehot shape: torch.Size([131, 4])
phi_angles shape: torch.Size([131, 1])
psi_angles shape: torch.Size([131, 1])
b_factors shape: torch.Size([131, 1])
Shape of labels: torch.Size([131])
esm2_embeddings shape: torch.Size([241, 1280])
ss_onehot shape: torch.Size([241, 4])
phi_angles shape: torch.Size([241, 1])
psi_angles shape: torch.Size([241, 1])
b_factors shape: torch.Size([241, 1])
Shape of labels: torch.Size([241])


100%|██████████| 300/300 [01:14<00:00,  4.04it/s]

esm2_embeddings shape: torch.Size([152, 1280])
ss_onehot shape: torch.Size([152, 4])
phi_angles shape: torch.Size([152, 1])
psi_angles shape: torch.Size([152, 1])
b_factors shape: torch.Size([152, 1])
Shape of labels: torch.Size([152])


### Create baseline GNN

In [30]:
class BindingSiteGCN(nn.Module):
    def __init__(self, node_dim=1287, edge_dim=2, hidden_dim=512):
        """
        A baseline GCN for binding site prediction.

        Args:
            node_dim (int): Dimension of node features (1288 in your case).
            edge_dim (int): Dimension of edge features (2 in your case).
            hidden_dim (int): Hidden dimension for GCN layers.
        """
        super(BindingSiteGCN, self).__init__()
        # GCN layers with edge features
        self.conv1 = GCNConv(node_dim, hidden_dim)  # Remove edge_dim
        self.conv2 = GCNConv(hidden_dim, 256)  # Remove edge_dim
        self.conv3 = GCNConv(256, 128)  # Remove edge_dim

        # Linear layer to process edge features
        self.edge_lin = nn.Linear(edge_dim, hidden_dim)

        # Final classifier
        self.fc = nn.Linear(128, 2)  # 2 classes: 0 (non-binding), 1 (binding)

        # Activation and dropout
        self.activation_func = nn.ReLU()
        self.dropout = nn.Dropout(0.1)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        # Get the number of nodes in the current batch

        # Process edge features separately
        edge_attr = self.edge_lin(edge_attr)

        # GCN layers
        # Pass num_nodes to gcn_norm
        x = self.conv1(x, edge_index)  # Removed edge_attr here, Added num_nodes
        x = self.activation_func(x)
        x = self.dropout(x)

        x = self.conv2(x, edge_index) # Removed edge_attr here, Added num_nodes
        x = self.activation_func(x)
        x = self.dropout(x)

        x = self.conv3(x, edge_index) # Removed edge_attr here, Added num_nodes
        x = self.activation_func(x)

        # Final classification
        x = self.fc(x)  # Shape: [num_residues, 2]

        return x  # Logits for each residue

In [31]:
train_graphs, val_graphs = train_test_split(
    train_graphs_data,
    test_size=0.05,
    random_state=42,
    shuffle=True
)

print(f"Number of training graphs: {len(train_graphs)}")
print(f"Number of validation graphs: {len(val_graphs)}")
print(f"Number of testing graphs: {len(test_graphs_data)}")

Number of training graphs: 962
Number of validation graphs: 51
Number of testing graphs: 300


In [32]:
for idx, train_graph in enumerate(train_graphs):
    if train_graph.x.shape[0] != train_graph.y.shape[0]:
        print(f"Abnormal at index: {idx}")
        print(f"Train graph's input shape: {train_graph['x'].shape[0]} and labels shape: {train_graph['y'].shape[0]}")

In [33]:
for idx, val_graph in enumerate(val_graphs):
    if val_graph.x.shape[0] != val_graph.y.shape[0]:
        print(f"Abnormal at index: {idx}")
        print(f"Train graph's input shape: {val_graph['x'].shape[0]} and labels shape: {val_graph['y'].shape[0]}")

### Model Training

In [34]:
# Create DataLoaders
batch_size = 16
train_loader = DataLoader(train_graphs, batch_size=batch_size, shuffle=True)
eval_loader = DataLoader(val_graphs, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_graphs_data, batch_size=batch_size, shuffle=False)

### Customized loss function

In [35]:
class WeightedCrossEntropyLoss(nn.Module):
    def __init__(self, pos_weight):
        super().__init__()
        self.pos_weight = pos_weight

    def forward(self, logits, labels):
        # Handle class imbalance with weighted loss
        weight = torch.tensor([1.0, self.pos_weight]).to(logits.device)  # [weight for class 0, weight for class 1]
        loss_fct = nn.CrossEntropyLoss(weight=weight)
        return loss_fct(logits, labels)

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super().__init__()
        self.alpha = alpha  # Weight for the positive class
        self.gamma = gamma  # Focusing parameter
        self.reduction = reduction

    def forward(self, logits, labels):
        # Compute the cross-entropy loss (without reduction)
        ce_loss = nn.functional.cross_entropy(logits, labels, reduction='none')

        # Compute the probability of the true class
        probs = torch.softmax(logits, dim=-1)
        true_probs = probs[torch.arange(probs.size(0), device=probs.device), labels]

        # Compute the focal loss term: (1 - p_t)^gamma
        focal_term = (1 - true_probs) ** self.gamma

        # Apply the alpha weighting
        alpha_weight = torch.where(labels == 1, self.alpha, 1.0 - self.alpha).to(logits.device)

        # Compute the focal loss
        loss = alpha_weight * focal_term * ce_loss

        # Apply reduction
        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss

class PositionAwareLoss(nn.Module):
    def __init__(self, pos_weight, position_weight, alpha=0.25, gamma=2.0):
        super().__init__()
        self.weighted_ce = WeightedCrossEntropyLoss(pos_weight)
        self.focal_loss = FocalLoss(alpha=alpha, gamma=gamma)
        self.position_weight = position_weight

    def forward(self, logits, labels, batch=None):
        # Compute the base loss (weighted cross-entropy + focal loss)
        ce_loss = self.weighted_ce(logits, labels)
        focal_loss = self.focal_loss(logits, labels)
        base_loss = ce_loss + focal_loss

        # Position-aware component
        probs = torch.softmax(logits, dim=-1)[:, 1]  # Get binding probabilities [total_num_nodes]
        position_loss = torch.tensor(0.0).to(logits.device)

        # Since we're using a GNN, we need to account for the graph structure
        # batch.batch indicates which nodes belong to which graph
        if batch is not None and batch.num_graphs == 1:  # Single graph per batch
            num_nodes = batch.num_nodes
            # Assume nodes are ordered sequentially (based on residue indices)
            # Penalize offset predictions by checking neighboring nodes
            for i in range(1, num_nodes - 1):
                # Encourage predictions to match true binding site positions
                if labels[i] == 1 or labels[i-1] == 1 or labels[i+1] == 1:
                    position_loss += torch.abs(probs[i] - (labels[i] == 1).float())

        return base_loss + self.position_weight * position_loss

In [36]:
model = BindingSiteGCN(node_dim=1287, edge_dim=2, hidden_dim=512).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

# class_weights = torch.tensor([0.2, 0.8]).to(device)
pos_weight = 5.0
position_weight = 0.75
focal_loss = FocalLoss(alpha=0.25, gamma=2.0)
# criterion = WeightedCrossEntropyLoss(pos_weight=pos_weight)
criterion = PositionAwareLoss(pos_weight=pos_weight, position_weight=position_weight, alpha=0.25, gamma=2.0)

In [37]:
def evaluate(model, data_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for batch in data_loader:
            batch = batch.to(device)
            out = model(batch)

            # Debug: Print shapes
            # print(f"Batch num_graphs: {batch.num_graphs}")
            # print(f"Batch num_nodes: {batch.num_nodes}")
            # print(f"Out shape: {out.shape}")
            # print(f"Labels shape: {batch.y.shape}")

            probs = torch.softmax(out, dim=1)[:, 1]
            preds = torch.argmax(out, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch.y.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')
    auc = roc_auc_score(all_labels, all_probs)

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "auc": auc
    }

In [38]:
gc.collect()  # Force garbage collection to potentially free up memory
# torch.cuda.empty_cache()  # Empty the CUDA cache
# model = model.half()  # Convert model parameters to half-precision
# torch.cuda.synchronize()

7111

In [39]:
num_epochs = 20
best_val_f1 = 0
best_model_state = None

model.train()
for epoch in range(num_epochs):
    total_train_loss = 0
    model.train()
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        print(f"Batch shape: {batch}")
        out = model(batch)
        print(f"Out shape: {out.shape}")
        print(f"Labels shape: {batch.y.shape}")
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)

    val_metrics = evaluate(model, eval_loader, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"  Training Loss: {avg_train_loss:.4f}")
    print(f"  Validation Precision: {val_metrics['precision']:.4f}")
    print(f"  Validation Recall: {val_metrics['recall']:.4f}")
    print(f"  Validation F1-Score: {val_metrics['f1']:.4f}")
    print(f"  Validation AUC-ROC: {val_metrics['auc']:.4f}")

    if val_metrics['f1'] > best_val_f1:
        best_val_f1 = val_metrics['f1']
        best_model_state = model.state_dict()
        # patience_counter = 0
        print("Best validation F1-score improved! Saving model state.")
    # else:
    #     patience_counter += 1
    #     print(f"  Patience counter: {patience_counter}/{patience}")

    # if patience_counter >= patience:
    #     print("Early stopping triggered!")
    #     break

Batch shape: DataBatch(x=[2868, 1287], edge_index=[2, 2868], edge_attr=[2868, 2], y=[2868], batch=[2868], ptr=[17])
Out shape: torch.Size([2868, 2])
Labels shape: torch.Size([2868])
Batch shape: DataBatch(x=[2249, 1287], edge_index=[2, 2249], edge_attr=[2249, 2], y=[2249], batch=[2249], ptr=[17])
Out shape: torch.Size([2249, 2])
Labels shape: torch.Size([2249])
Batch shape: DataBatch(x=[2617, 1287], edge_index=[2, 2617], edge_attr=[2617, 2], y=[2617], batch=[2617], ptr=[17])
Out shape: torch.Size([2617, 2])
Labels shape: torch.Size([2617])
Batch shape: DataBatch(x=[2890, 1287], edge_index=[2, 2890], edge_attr=[2890, 2], y=[2890], batch=[2890], ptr=[17])
Out shape: torch.Size([2890, 2])
Labels shape: torch.Size([2890])
Batch shape: DataBatch(x=[2584, 1287], edge_index=[2, 2584], edge_attr=[2584, 2], y=[2584], batch=[2584], ptr=[17])
Out shape: torch.Size([2584, 2])
Labels shape: torch.Size([2584])
Batch shape: DataBatch(x=[2961, 1287], edge_index=[2, 2961], edge_attr=[2961, 2], y=[2961]

In [40]:
if best_model_state is not None:
    model.load_state_dict(best_model_state)
model.eval()  # Set the model to evaluation mode
test_metrics = evaluate(model, test_loader, device)

In [41]:
print(test_metrics)

{'precision': 0.44131551901336075, 'recall': 0.3658204123360027, 'f1': 0.4000372647661636, 'auc': 0.7830528938493972}


In [42]:
all_preds, all_labels, all_probs = [], [], []

with torch.no_grad():
    for test_batch in test_loader:
        test_batch = test_batch.to(device)
        out = model(test_batch)

        # Debug: Print shapes
        # print(f"Batch num_graphs: {batch.num_graphs}")
        # print(f"Batch num_nodes: {batch.num_nodes}")
        # print(f"Out shape: {out.shape}")
        # print(f"Labels shape: {batch.y.shape}")

        probs = torch.softmax(out, dim=1)[:, 1]
        preds = torch.argmax(out, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch.y.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

In [43]:
all_test_prot_sequences = test_df['sequence'].tolist()
all_sequences = []
for prot_seq in all_test_prot_sequences:
    all_sequences.extend(list(prot_seq))

### Error analysis

In [44]:
from collections import Counter
from copy import deepcopy

test_aa_counter = Counter(all_sequences)
false_negatives_dict = {}

for prob, label, pred, acid in zip(all_probs, all_labels, all_preds, all_sequences):
    if label == 1 and pred == 0:

        if acid not in false_negatives_dict:
            false_negatives_dict[acid] = 1
        else:
            false_negatives_dict[acid] += 1

In [45]:
false_negatives_percentage_dict = deepcopy(false_negatives_dict)

for acid in false_negatives_percentage_dict:
    false_negatives_percentage_dict[acid] /= test_aa_counter[acid]

In [46]:
# print(false_negatives_dict)
print(false_negatives_percentage_dict)

{'V': 0.0060901339829476245, 'N': 0.005498533724340176, 'G': 0.006502890173410405, 'R': 0.005123825789923143, 'I': 0.009302325581395349, 'E': 0.008767881864328565, 'K': 0.007205764611689352, 'Q': 0.007077435470441299, 'L': 0.006469662528414058, 'P': 0.006191950464396285, 'M': 0.006300114547537228, 'T': 0.008741804558226662, 'A': 0.005516959542296689, 'F': 0.00468384074941452, 'D': 0.006968641114982578, 'S': 0.006612007405448294, 'H': 0.006417736289381563, 'Y': 0.007085498346717053, 'C': 0.002002002002002002, 'W': 0.005170630816959669}
